# Performance by product --> SoM Dashboard

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/Brands/SoM/data/'
CPG = 'danone' 
nielsen_cat = True #True JUST FOR RB_HYHO, DANONE, UNILEVER SO FAR!!!
past = False
SOM_CPG = CPG.title()

## Import

In [2]:
import os
import numpy as np
import pandas as pd
from IPython.display import display

pd.options.display.float_format = '{:.2f}'.format
print([x for x in os.listdir(BASE_DIR) if x.endswith(f'{CPG}.csv')])

['som_danone.csv', 'nielsen_danone.csv', 'products_danone.csv', 'nielsen_pred_danone.csv']


In [3]:
df = pd.read_csv(os.path.join(BASE_DIR,f'products_{CPG}.csv'),
                 sep='\t', encoding='UTF-16', low_memory=False)
df.sample(2)

,month,country,city name,store_name,category en,category_id,product id,product name,barcodes,package,...,sales,orders,avg ticket,users,units found,units replaced,units requested,found rate,fulfillment,frequency
13477,enero 2022,MX,Puebla,Chedraui,Yogurt,24,353551,Yogurt griego natural sin azúcar,{7501040095535},145 g,...,853,21,41,21,67,1,68,98.5%,100.0%,1.00
7775,enero 2022,MX,Guadalajara,City Market,Plant-based Beverages,1516,1253775,Alimento líquido de avena y linaza sin azúcar,{7502252484285},Cartón 946 ml,...,435,5,87,2,10,2,12,83.3%,100.0%,2.50


In [4]:
# df.rename(columns={'month  / year':'month  ','sales [currency]':'sales'}, inplace=True)

In [5]:
aux_ini = df.copy()
aux_ini['sales'] = aux_ini['sales'].astype(str).str.replace(',','').astype(float)
aux_ini = aux_ini.groupby('month  ')['sales'].sum().reset_index()
aux_ini

,month,sales
0,enero 2022,20213504.00


In [6]:
import re
nielsen = pd.read_csv(BASE_DIR + f'nielsen_{CPG}.csv',
                      sep='\t',encoding='utf-16'
                     ).drop_duplicates(subset=['UPC','PRODUCT ID'])
display(nielsen['UPC'].head())
nielsen['UPC'] = nielsen['UPC'].apply(lambda x: re.sub('[^\d]','',str(x)))
nielsen['UPC'] = nielsen['UPC'].astype(str).str.strip().str.split().str[0].fillna(0).astype(int)
display(nielsen['UPC'].head())
display(nielsen['UPC'].head())
nielsen.sample(2)

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: object

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: int64

0    75009076
1    75009083
2    75009090
3    75010065
4    75010751
Name: UPC, dtype: int64

,UPC,FORMATO,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,PRODUCT ID,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
4638,2000039090032,IND,PEPSICO,PEPSICO,BE LIGHT,BE LIGHT,1.50,BE LIGHT AGUA SABORIZADA PERA 1.5 LT IND W,NaN,BEBIDAS SABORIZADAS,AGUA SABORIZADA,NaN,NaN
2902,61500010524,PACK,DEMAS FABRICANTES,DEMAS FABRICANTES,STAR STRIPES,STAR STRIPES,0.50,STARS STRIPES AGUA PURIF 24PZS BOT PLA,NaN,AGUA NATURAL,AGUA NATURAL,NaN,NaN


## Functions

In [7]:
import time
import numpy as np
from IPython.lib.display import Audio

start = time.time()
def time_exp(x):
    minutes, seconds = np.floor(x/60), 60*(x/60-np.floor(x/60))
    print(f"{'{:.0f}'.format(minutes)} minutos con {'{:.2f}'.format(seconds)} segundos")
    
def tono(a = 1000, b = 700, play_time_seconds = 1, framerate = 4410):
    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)*np.pi
    return Audio(np.sin(a*t) + np.sin(b*t), rate = framerate, autoplay=True)

In [8]:
def limpio(data,month_col,to_drop):
    df = data.copy()
    df[['mes','anio']] = df[month_col].str.split(expand=True)
    meses = dict(zip(['enero','febrero','marzo','abril','mayo','junio','julio',
                      'agosto','septiembre','octubre','noviembre','diciembre'],range(1,13)))
    df['mes'] = df['mes'].replace(meses)
    df.drop(columns=to_drop,inplace=True)
    df.columns = [str(x).replace('.','').replace(' ','_').lower() for x in df.columns]
    for col in ['brand_filter','cpg_filter']:
        df[f'{col[:-7]}_id'] = df[col].apply(lambda x: ','.join(re.findall('\s\((\d+)\)',str(x))))
        df[f'{col[:-7]}_name'] = df[col].apply(lambda x: re.sub('\s\(\d+\)','',str(x)))
    return df

## Transform

### Barcodes

In [9]:
df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')
df['barcodes'] = df['barcodes'].str.split(',',n=1).fillna('.').apply(lambda x:x[0])
df['barcodes'].sample(4)

/var/folders/j4/1p33_bh96yn8pdz3b_5t80hm0000gn/T/ipykernel_4151/2845833164.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['barcodes'] = df['barcodes'].str.replace('{','').str.replace('}','')


11270    7501198353792
11892                .
793      7506443103535
8595     7501032396121
Name: barcodes, dtype: object

### Merge

In [10]:
df = df.astype(str).merge(nielsen.astype(str).rename(columns={'UPC':'barcodes',
                                                              'PRODUCT ID':'product id'}),
                          on=['barcodes'] if nielsen_cat else ['barcodes','product id'],
                          how='left' if nielsen_cat else 'left')
if nielsen_cat:
    df = df.rename(columns={'product id_x':'product_id'}).drop(columns='product id_y')

df.sample(2)

,month,country,city name,store_name,category en,category_id,product_id,product name,barcodes,package,...,PROVEEDOR GENERAL,PROVEEDOR ABIERTO,MARCA DETALLE,MARCA GENERICA,GRAMAJE,DESCRIPCION NIELSEN,SEGMENTO,SUBSEGMENTO 1,SUBSEGMENTO 2,SUBSEGMENTO 3
16195,enero 2022,MX,Querétaro,La Comer,Water,35,335771,Agua natural,758104001552,Botellas 6 x 1 L,...,DANONE,DANONE,BONAFONT,BONAFONT,1.0,BONAFONT SIX PACK PVP 5X6 1000ML C/U=6000ML NAL,AGUA NATURAL,AGUA NATURAL,nan,nan
8173,enero 2022,MX,Guadalajara,Fresko,Desserts & Refrigerated Bakery,22,345160,Gelatina de leche sabor coco,75042790,Vaso 100 g,...,DANONE,DANONE,DANETTE,DANETTE,0.1,DANETTE GELATINA DE LECHE COCO VASO 100 GR 1 PZA,GELATINAS,GELATINAS REGULARES,nan,nan


### CPG

In [11]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
enero 2022,20213504.00,20213504.00,0.00,0.00


In [12]:
prod_cpg = pd.read_csv(os.path.join(BASE_DIR,'pbbc.csv'),
                       #sep='\t',encoding='utf-16'
                      )
prod_cpg.sample(4)

,n,product_id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name
2034,1,94852,3600522289777,971,Cremas y tratamientos rostro,5171,Loréal Paris,60,L'Oreal
99851,1,2373572,16000515765,997,Barras energéticas y de granola,646,Nature Valley,193,General Mills
3223,1,98913,38000500381,981,Galletas dulces,640,Kellogg's,1696,Imalinx
94173,1,2026306,44600320779,1557,Carbón y encendido,16576,Kingsford,39,GIFAN


In [13]:
prod_cpg.drop_duplicates(('barcodes' if nielsen_cat else 'product_id'),inplace=True)
prod_cpg = prod_cpg.astype(str).rename(columns={'product_id':'product id'})
prod_cpg['cpg filter'] = prod_cpg['cpg_name']+" ("+prod_cpg['cpg_id'].astype(str)+")"
prod_cpg.sample(4)

,n,product id,barcodes,category_id,category_name,brand_id,brand_name,cpg_id,cpg_name,cpg filter
60753,1,391875,800443285190,230,Alimento seco,19027,Whole Hearted,583,Wholehearted,Wholehearted (583)
42853,1,322152,7506309864907,963,Desodorantes,447,Gillette,1,P&G MX,P&G MX (1)
40215,1,314626,7501036622424,991,Toallas de papel,5766,Regio,48,Essity,Essity (48)
73791,1,1109552,892383002333,228,Cuidados y mantenimiento,23541,Smartbones,1329,Spectrum Brands,Spectrum Brands (1329)


In [14]:
df = df.merge(prod_cpg[[('barcodes' if nielsen_cat else 'product id'),'cpg filter']],
              on=('barcodes' if nielsen_cat else 'product id'),how='left')

In [15]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('month  ')['sales'].sum().reset_index()
aux = aux_ini.copy()
aux = aux.set_index('month  ').join(aux_fin.set_index('month  '),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
enero 2022,20213504.00,20213504.00,0.00,0.00


### Limpio

In [16]:
df = limpio(df,'month  ',
            ['month  ','country ','avg ticket','found rate','fulfillment','frequency'])
df.sample(2)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id,cpg_name
16029,Querétaro,HEB,Yogurt,24,1005835,Yogurt estilo griego bebible con durazno,818290011756,207 ml,nan,Chobani (14849),...,GRIEGO,nan,nan,Chobani (284),1,2022,14849,Chobani,284,Chobani
277,Aguascalientes,HEB,Yogurt,24,335447,Yoghurt griego natural,7501032395582,Vaso 150 g,nan,Oikos (5502),...,GRIEGO,nan,nan,Danone (75),1,2022,5502,Oikos,75,Danone


In [17]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1
aux

,sales,sales_fin,dif,dif_p
month,,,,
ene,20213504.00,20213504.00,0.00,0.00


In [18]:
aux = df[df['formato'].astype(str)=='nan'].copy()
df = df[~(df['formato'].astype(str)=='nan')].copy()
null_fill = {'proveedor_general':'cpg_name',
             'proveedor_abierto':'cpg_name',
             'marca_detalle':'brand_name',
             'marca_generica':'brand_name',
             'gramaje':'package',
             'descripcion_nielsen':'product_name',
             'segmento':'category_en'}
for x,y in null_fill.items():
    aux.loc[:,x] = aux.loc[:,y]
    
df = df.append(aux,ignore_index=True)

In [19]:
for col in df.columns:
    df[col] = df[col].astype(str).str.replace('nan','')
    
df = df.drop(columns='cpg_name').fillna('')
df.sample()

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id
15349,Toluca,La Comer,Yogurt,24,335910,Queso petit suisse con fresa,7501032335052,Pack 4 x 84 g,,Danonino (4348),...,INFANTIL SOLIDO,INFANTIL,,,Danone (75),1,2022,4348,Danonino,75


In [20]:
brand_cols = ['formato','proveedor_general','proveedor_abierto','marca_detalle','marca_generica','gramaje','descripcion_nielsen','segmento','subsegmento_1','subsegmento_2','subsegmento_3']
display(df[brand_cols].sample())

for col in brand_cols:
    df[col] = df[col].str.strip().str.title()

display(df[brand_cols].sample())

,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
11976,LITRO,DANONE,,SILK,,0.946,SILK ALIMENTO LIQUIDO DE COCO S/AZUCAR CARTON ...,PLANT BASED,PLANT BASED,COCO,UHT


,formato,proveedor_general,proveedor_abierto,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3
9476,Ind,Demas Fabricantes,Demas Fabricantes,Natures Heart,Natures Heart,0.33,Natures Heart Agua De Coco 330Ml,Bebidas Saborizadas,Agua De Coco,,


In [21]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))
aux_fin

,mes,sales
0,ene,20213504.00


### Marca genérica

In [22]:
df.loc[df[df['marca_generica']==''].index, 'marca_generica'] = df.loc[df[df['marca_generica']==''].index, 'marca_detalle']
df[df['marca_generica']=='']

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_filter,...,segmento,subsegmento_1,subsegmento_2,subsegmento_3,cpg_filter,mes,anio,brand_id,brand_name,cpg_id


## Export

### Same product_id as past

In [23]:
print(len(df))
if past:
    som = pd.ExcelFile(os.path.join('/'.join(BASE_DIR.split('/')[:-2]),
                                    f'SoM_{SOM_CPG}.xlsx')).parse(sheet_name=0)
    df = df.merge(som.drop_duplicates('product_id')[['product_id']].reset_index(drop=True).astype(str))
    
print(len(df))

18987
18987


In [24]:
aux_fin = df.copy()
aux_fin['sales'] = aux_fin['sales'].astype(str).str.replace(',','').astype(float)
aux_fin = aux_fin.groupby('mes')['sales'].sum().reset_index()
aux_fin['mes'] = aux_fin['mes'].astype(int).map(dict(zip(range(1,13),
                                               ['ene','feb','mar','abr','may','jun',
                                                'jul','ago','sep','oct','nov','dic'])))

In [25]:
aux = aux_ini.copy()
aux['month  '] = aux['month  '].str.split().str[0].str[:3]
aux = aux.set_index('month  ').join(aux_fin.set_index('mes'),rsuffix='_fin')
aux['dif'] = aux['sales_fin'] - aux['sales']
aux['dif_p'] = aux['sales_fin'] / (aux['sales']+1e-10) - 1

### Ordered columns

In [26]:
export = df[['city_name', 'store_name', 'category_en', 'category_id', 'product_id',
             'product_name', 'barcodes', 'package', 'product_sku', 'brand_name',
             'sales', 'orders', 'users', 'units_found', 'units_replaced',
             'units_requested', 'formato', 'proveedor_general', 'proveedor_abierto',
             'marca_detalle', 'marca_generica', 'gramaje', 'descripcion_nielsen',
             'segmento', 'subsegmento_1', 'subsegmento_2', 'subsegmento_3',
             'mes', 'anio']].copy()
export.sort_values(['anio','mes'],inplace=True)
export.sample(4)

,city_name,store_name,category_en,category_id,product_id,product_name,barcodes,package,product_sku,brand_name,...,marca_detalle,marca_generica,gramaje,descripcion_nielsen,segmento,subsegmento_1,subsegmento_2,subsegmento_3,mes,anio
9923,Monterrey,HEB,Yogurt,24,336740,Yoghurt griego con kiwi mazana y jugo de uva,7501032395988,Vaso 150 g,,Oikos,...,Oikos,Oikos,0.15,Danone Oikos Griego Solido Frutos Verdes 150Gr...,Griego,Griego,,,1,2022
14847,Toluca,Chedraui,Yogurt,24,336476,Yoghurt doble cero sabor fresa,7501040090806,125 g,,Yoplait,...,Yoplait Doble Cero,Yoplait Doble Cero,0.125,Yoplait Light Fresa Yoghurt 125Gr,Light,Light,,,1,2022
14998,Toluca,City Market,Water,35,336849,Agua natural,758104000159,Botella 1.5 L,,Bonafont,...,Bonafont,Bonafont,1.5,Bonafont Natural Bot.Pla.1500 Ml Nal.,Agua Natural,Agua Natural,,,1,2022
13721,Querétaro,Soriana,Yogurt,24,326832,Alimento lácteo bebible ciruela pasa,7501032398149,Pack 6 x 225 g,,Activia,...,Activia,Activia,1.35,Danone Activia Ciruela Pasa 6Pza 225Grc/U,Salud Activa,Salud Digestiva,,,1,2022


In [27]:
export.to_csv(os.path.join(BASE_DIR,f'som_{CPG}.csv'),index=False,sep='\t',encoding='utf-16')

## End

In [28]:
aux

,sales,sales_fin,dif,dif_p
month,,,,
ene,20213504.00,20213504.00,0.00,0.00


In [29]:
time_exp(time.time() - start)
tono()

0 minutos con 2.66 segundos
